In [49]:
import geopandas as gpd
import pandas as pd
import fiona as fi
import glob
import rasterio
from rasterstats import zonal_stats
from shapely.geometry import Point, LineString, Polygon

In [50]:
#read files in data folder
glob.glob(r'C:\Users\arom\Documents\School\Grad School\Fall 2020\Automation\week3\Lab2\lab2_data\data\*\*')

['C:\\Users\\arom\\Documents\\School\\Grad School\\Fall 2020\\Automation\\week3\\Lab2\\lab2_data\\data\\agriculture\\GLOBCOVER_2004_lab2.tif',
 'C:\\Users\\arom\\Documents\\School\\Grad School\\Fall 2020\\Automation\\week3\\Lab2\\lab2_data\\data\\agriculture\\GLOBCOVER_2004_lab2.tif.vat.cpg',
 'C:\\Users\\arom\\Documents\\School\\Grad School\\Fall 2020\\Automation\\week3\\Lab2\\lab2_data\\data\\agriculture\\GLOBCOVER_2004_lab2.tif.vat.dbf',
 'C:\\Users\\arom\\Documents\\School\\Grad School\\Fall 2020\\Automation\\week3\\Lab2\\lab2_data\\data\\agriculture\\GLOBCOVER_2009_lab2.tif',
 'C:\\Users\\arom\\Documents\\School\\Grad School\\Fall 2020\\Automation\\week3\\Lab2\\lab2_data\\data\\agriculture\\GLOBCOVER_2009_lab2.tif.aux.xml',
 'C:\\Users\\arom\\Documents\\School\\Grad School\\Fall 2020\\Automation\\week3\\Lab2\\lab2_data\\data\\agriculture\\GLOBCOVER_2009_lab2.tif.vat.cpg',
 'C:\\Users\\arom\\Documents\\School\\Grad School\\Fall 2020\\Automation\\week3\\Lab2\\lab2_data\\data\\agricu

In [51]:
#create list of files
file_names = [r'C:\Users\arom\Documents\School\Grad School\Fall 2020\Automation\week3\Lab2\lab2_data\data\districts\district01.txt', r'C:\Users\arom\Documents\School\Grad School\Fall 2020\Automation\week3\Lab2\lab2_data\data\districts\district05.txt', r'C:\Users\arom\Documents\School\Grad School\Fall 2020\Automation\week3\Lab2\lab2_data\data\districts\district06.txt']
#create empty dictionary
dictionary = {'district': [], 'num_coords':[], 'geometry': []}
#run for loop to read files in list, use the coordinates from each file to create a polygon
#create variable that has the amount of coordinate pairs, use slicing to isolate each district name
#append the empty dictionary with the output generated in the for loop
for i in file_names:
    x = pd.read_csv(i , delim_whitespace = True)
    coords = list(zip(x['X'], x['Y']))
    poly = Polygon(coords)
    num_coords = len(coords)
    district = i[-14: -4]
    dictionary['district'].append(district)
    dictionary['num_coords'].append(num_coords)
    dictionary['geometry'].append(poly)

#create geodataframe and project    
districts_gdf = gpd.GeoDataFrame.from_dict(dictionary)
districts_gdf.crs = {'init': 'epsg:4326'}

#create shapefile
districts_gdf.to_file(driver='ESRI Shapefile', filename ='districts_gdf.shp')

C:\Users\arom\anaconda3\envs\geog5092_37\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [52]:
dictionary

{'district': ['district01', 'district05', 'district06'],
 'num_coords': [1847, 566, 1223],
 'geometry': [<shapely.geometry.polygon.Polygon at 0x1b132b8c888>,
  <shapely.geometry.polygon.Polygon at 0x1b132ba6988>]}

In [53]:
#use zonal stats to obtain total pixel count and agricultural pixel count for 2004 and 2009
zonal04 = zonal_stats(r"C:\Users\arom\Documents\School\Grad School\Fall 2020\Automation\week3\Lab2\districts_gdf.shp", r"C:\Users\arom\Documents\School\Grad School\Fall 2020\Automation\week3\Lab2\lab2_data\data\agriculture\GLOBCOVER_2004_lab2.tif",
            stats="count min mean max median", categorical=True)
zonal09 = zonal_stats(r"C:\Users\arom\Documents\School\Grad School\Fall 2020\Automation\week3\Lab2\districts_gdf.shp", r"C:\Users\arom\Documents\School\Grad School\Fall 2020\Automation\week3\Lab2\lab2_data\data\agriculture\GLOBCOVER_2009_lab2.tif",
            stats="count min mean max median", categorical=True)


In [54]:
zonal04

[{0: 53877,
  1: 35922,
  'min': 0.0,
  'max': 1.0,
  'mean': 0.40002672635552733,
  'count': 89799,
  'median': 0.0},
 {0: 11946,
  1: 6639,
  'min': 0.0,
  'max': 1.0,
  'mean': 0.35722356739305894,
  'count': 18585,
  'median': 0.0},
 {0: 46830,
  1: 39629,
  'min': 0.0,
  'max': 1.0,
  'mean': 0.45835598376108905,
  'count': 86459,
  'median': 0.0}]

In [55]:
zonal09

[{0: 40312,
  1: 49487,
  'min': 0.0,
  'max': 1.0,
  'mean': 0.5510863149923718,
  'count': 89799,
  'median': 1.0},
 {0: 12136,
  1: 6449,
  'min': 0.0,
  'max': 1.0,
  'mean': 0.34700026903416736,
  'count': 18585,
  'median': 0.0},
 {0: 47274,
  1: 39185,
  'min': 0.0,
  'max': 1.0,
  'mean': 0.453220601672469,
  'count': 86459,
  'median': 0.0}]

In [56]:
p_totals = [89799, 18585, 86459]
p_ag04 = [35922, 6639, 39629] #agricultural pixels in 2004
p_ag09 = [49487, 6449, 39185] #agricultural pixels in 2009

In [57]:
#to get percentage of agricultural pixels, divide agricultural pixels by total pixels, then mulitply by 100
percent_agriculture_04 = [x / y * 100 for x,y in zip(p_ag04, p_totals)]
percent_agriculture_09 = [x / y * 100 for x,y in zip(p_ag09, p_totals)]


In [58]:
percent_agriculture_04

[40.002672635552734, 35.722356739305894, 45.835598376108905]

In [59]:
percent_agriculture_09

[55.108631499237184, 34.700026903416735, 45.3220601672469]

In [60]:
final = {'District':['01', '05', '06' ],'2004%':['40.00', '35.72', '45.84'],'2009%':['55.11', '34.70', '45.32']}
final = pd.DataFrame(final)
print(final)

  District  2004%  2009%
0       01  40.00  55.11
1       05  35.72  34.70
2       06  45.84  45.32
